Download Data

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"illidan7","key":"e84c6f64f4ca91fc6b8c48139bf32342"}'}

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!ls -hlt

total 8.0K
-rw-r--r-- 1 root root   64 Feb 20 15:21 kaggle.json
drwxr-xr-x 1 root root 4.0K Feb 16 16:35 sample_data


In [ ]:
!kaggle kernels list --user Illidan7 --sort-by dateRun

ref                                               title                                      author    lastRunTime          totalVotes  
------------------------------------------------  -----------------------------------------  --------  -------------------  ----------  
illidan7/pytorchmodelsubmission                   PytorchModelSubmission                     Illidan7  2021-02-20 04:43:19           0  
illidan7/pytorch-optuna-tutorial                  Pytorch-Optuna-Tutorial                    Illidan7  2021-02-14 00:57:33           0  
illidan7/notebook7e318a04ea                       notebook7e318a04ea                         Illidan7  2021-02-07 14:06:43           0  
illidan7/simple-pytorch-tensorflow-mlp-starter    Simple Pytorch Tensorflow MLP - Starter    Illidan7  2021-02-07 06:45:54           0  
illidan7/blending-tensorflow-and-pytorch-starter  Blending tensorflow and pytorch - Starter  Illidan7  2021-02-05 17:04:32           0  
illidan7/vegasml                         

In [ ]:
!kaggle competitions download -c jane-street-market-prediction

  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 60.2MB/s]
  0% 0.00/59.0 [00:00<?, ?B/s]
100% 59.0/59.0 [00:00<00:00, 51.9kB/s]
  0% 0.00/108k [00:00<?, ?B/s]
100% 108k/108k [00:00<00:00, 112MB/s]
  0% 0.00/23.3k [00:00<?, ?B/s]
100% 23.3k/23.3k [00:00<00:00, 20.5MB/s]
 30% 5.00M/16.4M [00:00<00:00, 28.2MB/s]
100% 16.4M/16.4M [00:00<00:00, 65.1MB/s]
100% 2.60G/2.61G [00:44<00:00, 52.8MB/s]
100% 2.61G/2.61G [00:44<00:00, 63.5MB/s]


In [ ]:
!unzip train.csv.zip -d .

Archive:  train.csv.zip
  inflating: ./train.csv             


In [ ]:
!ls -hlt

total 8.4G
-rw-r--r-- 1 root root 2.7G Feb 20 15:22 train.csv.zip
-rw-r--r-- 1 root root  17M Feb 20 15:21 example_test.csv.zip
-rw-r--r-- 1 root root  24K Feb 20 15:21 features.csv
-rw-r--r-- 1 root root 109K Feb 20 15:21 example_sample_submission.csv
-rw-r--r-- 1 root root   59 Feb 20 15:21 __init__.py
-rw-r--r-- 1 root root 442K Feb 20 15:21 competition.cpython-37m-x86_64-linux-gnu.so
-rw-r--r-- 1 root root   64 Feb 20 15:21 kaggle.json
drwxr-xr-x 1 root root 4.0K Feb 16 16:35 sample_data
-rw-r--r-- 1 root root 5.8G Dec 17 23:51 train.csv


In [ ]:
!mkdir input/
!mv *.csv input/
!mv *.csv.zip input/
!ls -hlt

total 460K
drwxr-xr-x 2 root root 4.0K Feb 20 15:24 input
-rw-r--r-- 1 root root   59 Feb 20 15:21 __init__.py
-rw-r--r-- 1 root root 442K Feb 20 15:21 competition.cpython-37m-x86_64-linux-gnu.so
-rw-r--r-- 1 root root   64 Feb 20 15:21 kaggle.json
drwxr-xr-x 1 root root 4.0K Feb 16 16:35 sample_data


In [ ]:
!ls -hlt input/

total 8.4G
-rw-r--r-- 1 root root 2.7G Feb 20 15:22 train.csv.zip
-rw-r--r-- 1 root root  17M Feb 20 15:21 example_test.csv.zip
-rw-r--r-- 1 root root  24K Feb 20 15:21 features.csv
-rw-r--r-- 1 root root 109K Feb 20 15:21 example_sample_submission.csv
-rw-r--r-- 1 root root 5.8G Dec 17 23:51 train.csv


In [ ]:
!mkdir working/

Data load and cleaning

In [ ]:
!pip install datatable 
!pip install optuna

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import optuna
import datatable as dt
import os
import pickle
import random

In [ ]:
def df_column_types(df, cardinality_cutoff=10):

    # Select categorical columns with low cardinality
    low_card_cols = [cname for cname in df.columns if
                        df[cname].nunique() <= cardinality_cutoff and
                        (df[cname].dtype == "object" or df[cname].dtype == "category" or df[cname].dtype == "bool" or "code" in cname)]

    # Select categorical columns with high cardinality
    high_card_cols = [cname for cname in df.columns if
                        df[cname].nunique() > cardinality_cutoff and
                        df[cname].dtype == "object"]

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    # Select numerical columns
    numerical_cols = [cname for cname in df.columns if
                    df[cname].dtype in numerics and "code" not in cname]

    return low_card_cols, high_card_cols, numerical_cols

In [ ]:
def pickle_dump(path, saveobj):
    import pickle
    filehandler = open(path,"wb")
    pickle.dump(saveobj,filehandler)
    print("File pickled")
    filehandler.close()

In [ ]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0)) 

1
Tesla T4


In [ ]:
import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

Torch 1.7.0+cu101 CUDA 10.1
Device: cuda:0


In [ ]:
print("Loading data")

df_dt = dt.fread("./input/train.csv")
df = df_dt.to_pandas()

features = [c for c in df.columns if 'feature' in c]

Loading data


In [ ]:
print("Filling missing values")

f_mean = df[features[1:]].mean()

df  = df.loc[df.weight > 0].reset_index(drop=True)
df[features[1:]] = df[features[1:]]. fillna(f_mean)
df['action'] = (df['resp'] > 0).astype('int')

df = df.astype({c: np.float32 for c in df.select_dtypes(include='float64').columns})

f_mean = f_mean.values
np.save('./working/f_mean.npy', f_mean)

Filling missing values


In [ ]:
print("Robust Scaling")

df['feature_0'] = df['feature_0'].astype('category')

low_card_cols, high_card_cols, numerical_cols = df_column_types(df[features])

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(quantile_range=(5,95))
scaler.fit(df[numerical_cols])

df = pd.concat([df[[col for col in df.columns if col not in numerical_cols]].reset_index(drop=True),
                     pd.DataFrame(scaler.transform(df[numerical_cols]), columns=df[numerical_cols].columns)], axis=1)

pickle_dump('./working/scaler.pkl', scaler)

Robust Scaling
File pickled


In [ ]:
pickle.dump(df, open('./working/train.csv.pandas.pickle', 'wb'))
df.shape

(1981287, 139)

In [ ]:
!nvidia-smi

Sat Feb 20 02:27:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
del df
import gc
gc.collect()

31

In [ ]:
%%time
train_pickle_file = './working/train.csv.pandas.pickle'
df = pickle.load(open(train_pickle_file, 'rb'))
df.shape

CPU times: user 278 ms, sys: 410 ms, total: 688 ms
Wall time: 697 ms


In [ ]:
%%time
# GroupTimeSeriesSplit.py
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

            
def create_folds(df):
#     df.loc[:,"kfold"] = -1
    
    for fold,(train_idx, valid_idx) in enumerate(GroupTimeSeriesSplit().split(df, groups=df['date'])):
        train_temp = df.loc[train_idx,:]
        valid_temp = df.loc[valid_idx,:]
        train_temp.to_csv(f"./working/train_fold{fold+1}.csv", index=False)
        valid_temp.to_csv(f"./working/valid_fold{fold+1}.csv", index=False)
        
    print("Folds created")


temp = df[['ts_id','date']].copy()
create_folds(temp)

Folds created
CPU times: user 46.3 s, sys: 616 ms, total: 46.9 s
Wall time: 47 s


In [ ]:
del df
import gc
gc.collect()

0

### Model training

In [ ]:
!pip install datatable 
!pip install optuna

import numpy as np
import pandas as pd

import optuna
import torch
import datatable as dt
import os
import pickle
import random

     |██████████████████████████▉     | 70.3MB 4.7MB/s eta 0:00:03

In [ ]:
!ls -hlt ./working/

total 1.1G
-rw-r--r-- 1 root root  92K Feb 20 17:10 model_1.bin
-rw-r--r-- 1 root root  25K Feb 20 17:06 model_5.bin
-rw-r--r-- 1 root root  25K Feb 20 17:05 model_4.bin
-rw-r--r-- 1 root root  25K Feb 20 16:51 model_3.bin
-rw-r--r-- 1 root root  25K Feb 20 16:40 model_2.bin
-rw-r--r-- 1 root root 4.3M Feb 20 16:23 valid_fold5.csv
-rw-r--r-- 1 root root  18M Feb 20 16:23 train_fold5.csv
-rw-r--r-- 1 root root 4.0M Feb 20 16:23 valid_fold4.csv
-rw-r--r-- 1 root root  14M Feb 20 16:23 train_fold4.csv
-rw-r--r-- 1 root root 3.6M Feb 20 16:22 valid_fold3.csv
-rw-r--r-- 1 root root 9.7M Feb 20 16:22 train_fold3.csv
-rw-r--r-- 1 root root 3.3M Feb 20 16:22 valid_fold2.csv
-rw-r--r-- 1 root root 6.4M Feb 20 16:22 train_fold2.csv
-rw-r--r-- 1 root root 2.7M Feb 20 16:22 valid_fold1.csv
-rw-r--r-- 1 root root 3.7M Feb 20 16:22 train_fold1.csv
-rw-r--r-- 1 root root 1.1G Feb 20 16:22 train.csv.pandas.pickle
-rw-r--r-- 1 root root 2.0K Feb 20 16:22 scaler.pkl
-rw-r--r-- 1 root root  644 Feb 20 16

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=42)

In [ ]:
# utils.py
import torch
import torch.nn as nn

class JaneStreetDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets.reshape(-1, 1)
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, item):
        return {
            "x": torch.tensor(self.features[item, :], dtype=torch.float),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float)
        }

    
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.device = device
        self.optimizer = optimizer
    
    @staticmethod
    def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)
    
    def train(self, data_loader):
        self.model.train()
        final_loss=0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss/ len(data_loader)
    
    def evaluate(self, data_loader):
        self.model.eval()
        final_loss=0
        for data in data_loader:
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            final_loss += loss.item()
        return final_loss/ len(data_loader)

    
class Model(nn.Module):
    def __init__(self, nfeatures, ntargets, nlayers, hidden_size, dropout):
        super().__init__()
        layers = []
        for _ in range(nlayers):
            if len(layers) == 0:
                layers.append(nn.Linear(nfeatures, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, ntargets))
        self.model = nn.Sequential(*layers)
    
    
    def forward(self, x):
        return self.model(x)


In [ ]:
nlayers=5
hidden_size=256

hidden_neurons = []

n=5
while n > 0:
  mid=True if (nlayers//2)+1 == n else False
  if nlayers % 2 == 0:
    pass
  else:
    if mid:
      hidden_neurons.append(hidden_size)
      n-=1
      continue
    comp = nlayers - (n//2) 
    hidden_neurons.append(hidden_size//comp)
    n-=1


In [ ]:
hidden_neurons

[85, 85, 256, 64, 51]

In [ ]:
# train.py
import torch
import pandas as pd
import numpy as np
import datatable as dt
import pickle

DEVICE = torch.device('cuda:0')
EPOCHS = 100

def run_training(fold, params, save_model=False):
    train_pickle_file = './working/train.csv.pandas.pickle'
    df = pickle.load(open(train_pickle_file, 'rb'))
    
    print("Dataset loaded")
    
    # Read in training folds
    train_fold = pd.read_csv(f"./working/train_fold{fold+1}.csv")
    valid_fold = pd.read_csv(f"./working/valid_fold{fold+1}.csv")

    feature_cols = [c for c in df.columns if "feature" in c]
    
    # Train and validation folds
    train_df = pd.merge(df, train_fold, on="ts_id", how="inner")
    valid_df = pd.merge(df, valid_fold, on="ts_id", how="inner")
    
    print("Folds loaded")
    
    xtrain = train_df[feature_cols].to_numpy()
    ytrain = train_df["action"].to_numpy()
    
    xvalid = valid_df[feature_cols].to_numpy()
    yvalid = valid_df["action"].to_numpy()
    
    train_dataset = JaneStreetDataset(features=xtrain, targets=ytrain)
    valid_dataset = JaneStreetDataset(features=xvalid, targets=yvalid)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4096, num_workers=4)
    valid_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4096, num_workers=4)
    
    model = Model(
        nfeatures=np.shape(xtrain)[1], 
        ntargets=1, 
        nlayers=params['num_layers'], 
        hidden_size=params['hidden_size'],
        dropout=params['dropout']
    )
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    eng = Engine(model, optimizer, device=DEVICE)
    
    best_loss = np.inf
    early_stopping_iter = 5
    early_stopping_counter = 0
    
    print("Model training begins")
    
    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(valid_loader)
        print(f"{fold}, {epoch}, {train_loss}, {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), f"./working/model_{fold+1}.bin")
        else:
            early_stopping_counter+=1
        
        if early_stopping_counter > early_stopping_iter:
            break
    
    return best_loss



def objective(trial):
    params = {
        "num_layers": trial.suggest_int("num_layers", 1, 3),
        "hidden_size": trial.suggest_int("hidden_size", 16, 512),
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-2)
    }
    all_losses = []
    for f_ in range(5):
        temp_loss = run_training(f_, params, save_model=True)
        all_losses.append(temp_loss)
    
    return np.mean(all_losses)

ModuleNotFoundError: ignored

Run Optuna Parameter search

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best trial:")
trial_ = study.best_trial

print(trial_.values)
print(trial_.params)

scores = 0
for j in range(5):
    scr = run_training(j, trial_.params, save_model=True)
    scores+=scr

print(scores/5)

[I 2021-02-20 17:12:59,525] A new study created in memory with name: no-name-fc7d26c5-b49a-4ae8-9052-4d7a8b022ad2


Dataset loaded
Folds loaded
Model training begins
0, 0, 0.70418310834437, 0.6971893146329996
0, 1, 0.6990697803545971, 0.6945796438625881
0, 2, 0.6970937245962571, 0.6935100993331598
0, 3, 0.6962012715485631, 0.6926304734483058
0, 4, 0.6953449778410853, 0.6922015523423954
0, 5, 0.6946916951208698, 0.6915143795159399
0, 6, 0.6942749716797654, 0.6913513644617431
0, 7, 0.6940952940863006, 0.6907652367134484
0, 8, 0.6935710499481279, 0.69099922934357
0, 9, 0.6930077532116248, 0.6908180652832498
Dataset loaded
Folds loaded
Model training begins
1, 0, 0.7053235343524388, 0.6939321674915574
1, 1, 0.6998922602730508, 0.6925650605503817
1, 2, 0.6981402972470159, 0.6918784013446073
1, 3, 0.6973283964654674, 0.6914176119040258
1, 4, 0.6963621922161268, 0.6911784543754151
1, 5, 0.6956735234823286, 0.6907628711706363
1, 6, 0.6951997591101605, 0.6905842675185352
1, 7, 0.6946855664993665, 0.6903563711954199
1, 8, 0.69402914017624, 0.6902328681501543
1, 9, 0.6939223312443088, 0.6900617091552071
Datase

[I 2021-02-20 17:50:39,236] Trial 0 finished with value: 0.6905569239105374 and parameters: {'num_layers': 2, 'hidden_size': 257, 'dropout': 0.16941247127490633, 'learning_rate': 2.7086490992178346e-05}. Best is trial 0 with value: 0.6905569239105374.


4, 9, 0.6933721909063116, 0.6906341731850871
Dataset loaded
Folds loaded
Model training begins
0, 0, 0.7246920369109329, 0.706326043119236
0, 1, 0.7194546424612707, 0.7017281590675821
0, 2, 0.7175446590598749, 0.6989725657871791
0, 3, 0.7161290067799237, 0.6972217316530189
0, 4, 0.7137845608652854, 0.6960805417323599
0, 5, 0.713078459306639, 0.6953138295485048
0, 6, 0.7127678747079811, 0.694736897945404
0, 7, 0.711796233240439, 0.6942909919485754
0, 8, 0.7116225939624163, 0.6939350859243043
0, 9, 0.7107661999001795, 0.6936269968139882
Dataset loaded
Folds loaded
Model training begins
1, 0, 0.723058637624942, 0.7053882709201078
1, 1, 0.7181938997707011, 0.7007652261982793
1, 2, 0.7148143027139746, 0.6979350388420295
1, 3, 0.7128842158347183, 0.6961992497769942
1, 4, 0.7115141804914297, 0.6951564542995476
1, 5, 0.7102674053322454, 0.694517549520694
1, 6, 0.7098911556397905, 0.6940630596616993
1, 7, 0.7083090254979104, 0.6937020872690662
1, 8, 0.7083363892128749, 0.693436160220863
1, 9, 0

[I 2021-02-20 18:27:23,083] Trial 1 finished with value: 0.6931904842295029 and parameters: {'num_layers': 1, 'hidden_size': 176, 'dropout': 0.484000147682183, 'learning_rate': 9.292696245366525e-06}. Best is trial 0 with value: 0.6905569239105374.


...

4, 9, 0.6914467903865775, 0.6912716790504262
Dataset loaded
Folds loaded
Model training begins
0, 0, 0.7261203156442059, 0.7241943831346473
0, 1, 0.7214140186504442, 0.7178131664285854
0, 2, 0.718271653871147, 0.7129568281222363
0, 3, 0.7151787603388027, 0.7092646561106857
0, 4, 0.712822322334562, 0.706165532676541
0, 5, 0.7119552821529155, 0.7036603695275833
0, 6, 0.7098288079913782, 0.7018720027135343
0, 7, 0.7098820331145306, 0.7004393910875126
0, 8, 0.7085669959078029, 0.699302077293396
0, 9, 0.7078411074317231, 0.6981779379504067
Dataset loaded
Folds loaded
Model training begins
1, 0, 0.7181906555750355, 0.7016008814669544
1, 1, 0.7140487043753915, 0.6990453064071466
1, 2, 0.7121970257403688, 0.6975118928814527
1, 3, 0.7105604399065053, 0.6964803223284135
1, 4, 0.7091552515207611, 0.6959211389470544
1, 5, 0.7087581109556352, 0.6953128080190338
1, 6, 0.7079286353188272, 0.6949242149820979
1, 7, 0.7072779225266498, 0.694500842449828
1, 8, 0.7064839931008238, 0.6942198128433701
1, 9,

[I 2021-02-20 21:51:03,772] Trial 7 finished with value: 0.6948918532106045 and parameters: {'num_layers': 2, 'hidden_size': 438, 'dropout': 0.28919384887382393, 'learning_rate': 1.0963895215599248e-06}. Best is trial 4 with value: 0.6894613431471952.


4, 9, 0.7039281291707518, 0.6940981005956679
Dataset loaded
Folds loaded
Model training begins
0, 0, 0.7131245963427485, 0.6913726524430879
0, 1, 0.7001157926053417, 0.6911732037456668
0, 2, 0.695426628297689, 0.6908193784100669
0, 3, 0.6933160558038828, 0.6904465720361593
0, 4, 0.6923333035439861, 0.6902550556221787
0, 5, 0.6916733511856624, 0.6899756545923195
0, 6, 0.6912391252663671, 0.6898329233636662
0, 7, 0.6909304328110754, 0.6896867478380397
0, 8, 0.6907849476045492, 0.689581784666801
0, 9, 0.6904332923645876, 0.6894532606309774
Dataset loaded
Folds loaded
Model training begins
1, 0, 0.7068014996392387, 0.6937266936213333
1, 1, 0.6963604059278595, 0.6920210485873015
1, 2, 0.6934406872121444, 0.6916408905331393
1, 3, 0.6923739410335232, 0.691365250889559
1, 4, 0.69181272739209, 0.6910226589404278
1, 5, 0.6915495884344445, 0.6908608978579504
1, 6, 0.6913097796973234, 0.6907175886705055
1, 7, 0.6912568885347118, 0.6905815501390777
1, 8, 0.691004863066703, 0.6904263418653737
1, 9, 

[I 2021-02-20 22:28:07,871] Trial 8 finished with value: 0.6906097515791125 and parameters: {'num_layers': 3, 'hidden_size': 207, 'dropout': 0.5460692690641846, 'learning_rate': 0.000369120449036248}. Best is trial 4 with value: 0.6894613431471952.


4, 9, 0.6914790548043808, 0.6918912896044969
Dataset loaded
Folds loaded
Model training begins
0, 0, 0.7036668366315414, 0.6935818049372459
0, 1, 0.6976624112956378, 0.6915973528307311
0, 2, 0.6957718152172712, 0.6906720673551365
0, 3, 0.6939752813504667, 0.690234007884045
0, 4, 0.6935806085868758, 0.6898201935145319
0, 5, 0.6926122113149993, 0.6896167166379034
0, 6, 0.6920616912598513, 0.6895606882718145
0, 7, 0.6916561832233351, 0.6894211361602861
0, 8, 0.691410801240376, 0.6893303491631333
0, 9, 0.6910170049083476, 0.6892916700061487
Dataset loaded
Folds loaded
Model training begins
1, 0, 0.7057050251812669, 0.6935625861150138
1, 1, 0.6987977064914584, 0.69230398109981
1, 2, 0.6962454763258467, 0.691611289237597
1, 3, 0.6943518382421932, 0.6913191224477306
1, 4, 0.6935414879218392, 0.6910418072102232
1, 5, 0.6928437740906425, 0.6908758501088397
1, 6, 0.6924868574053604, 0.6908204388914642
1, 7, 0.6919871224379688, 0.690694915211719
1, 8, 0.6917359851161886, 0.6906491340317341
1, 9, 

In [ ]:
print(trial_.values)
print(trial_.params)

NameError: ignored

Train individual model

In [ ]:
params = {'num_layers': 3, 'hidden_size': 336, 'dropout': 0.20201528699241103, 'learning_rate': 4.94148370092176e-05}

scores = 0
for j in range(5):
    scr = run_training(j, params, save_model=True)
    scores+=scr

print(scores/5)

Dataset loaded
Folds loaded
Model training begins
0, 0, 0.7052482536860875, 0.6920889774147345
0, 1, 0.6950483164008783, 0.6912245884233591
0, 2, 0.692166198881305, 0.69032499011682
0, 3, 0.6914172464487504, 0.690012742670215
0, 4, 0.6909937481490933, 0.689810322863715
0, 5, 0.6905289067297565, 0.6896148019907425
0, 6, 0.6903887974972628, 0.6894238773657351
0, 7, 0.6902807805002952, 0.6892712438593105
0, 8, 0.6899256408214569, 0.6891119498379377
0, 9, 0.6897813562227755, 0.6889638712211531
0, 10, 0.689646800561827, 0.68883387531553
0, 11, 0.6895597589259245, 0.6886984784992374
0, 12, 0.689414489634183, 0.6887135815863706
0, 13, 0.6889850253961525, 0.6885066993382513
0, 14, 0.6888341861111777, 0.6884232169511367
0, 15, 0.688794342838988, 0.6885153158586852
0, 16, 0.6885442879735207, 0.6882855423859188
0, 17, 0.6884289487284057, 0.6881188087317408
0, 18, 0.6881218382290432, 0.6882034102264716
0, 19, 0.6879448215572201, 0.6880252197080728
0, 20, 0.6877479334266818, 0.6879950019778037
0, 2